In [1]:
import pandas as pd

# Data Staging: 
## Extract, Transform and Load (ETL)

## Conceptual model:

<img src = "4142DIAGRAM.png" style ="width: 800px">

# Extract

In [2]:
df= pd.read_csv('./healthcare-dataset-stroke-data.csv')

In [3]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


## Changing "Residence_type" column name to "residence_type" to ensure uniformity with the data.

In [4]:
df = df.rename({'Residence_type' : 'residence_type'},axis = 'columns')
df = df.replace( {'formerly smoked':'Formerly smoked', 'never smoked': 'Never smoked', 'smokes':'Smokes', 'children': 'Children'})

In [5]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,Formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,Never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,Never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,Smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,Never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,Never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,Never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,Never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,Formerly smoked,0


## Finding null values in each column

In [6]:
df.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

Dropping missing BMI values 

In [10]:
df = df.dropna()

## Start Binning Data 

# Making sample data before transforming entire dataset

Starting with binning age group

In [8]:
df['age_group'] = pd.cut(df['age'], bins=[0,10,20,30,40,50,60,70,80,90,100], right=False, labels= ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90+'])

C:\Users\hamid\AppData\Local\Temp\ipykernel_45668\2082527355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age_group'] = pd.cut(df['age'], bins=[0,10,20,30,40,50,60,70,80,90,100], right=False, labels= ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90+'])


Binning BMI, have not accounted for Null values yet.

In [13]:
df['bmi_category'] = pd.cut(df['bmi'],bins=[0,18.5,25,30,40,100], right=False, labels= ['Underweight','Normal weight', 'Obese class I', 'Obese class II', 'Morbidly obese'])

Binning glucose

In [14]:
df['glucose_range'] = pd.cut(df['avg_glucose_level'],bins=[0,200,500], right=False, labels= ['Non-diabetic','Diabetic'])

In [15]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_group,bmi_category,glucose_range
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,Formerly smoked,1,60-69,Obese class II,Diabetic
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,Never smoked,1,80-89,Obese class II,Non-diabetic
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,Smokes,1,40-49,Obese class II,Non-diabetic
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,Never smoked,1,70-79,Normal weight,Non-diabetic
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,Formerly smoked,1,80-89,Obese class I,Non-diabetic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,14180,Female,13.0,0,0,No,Children,Rural,103.08,18.6,Unknown,0,10-19,Normal weight,Non-diabetic
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,Never smoked,0,80-89,Morbidly obese,Non-diabetic
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,Never smoked,0,30-39,Obese class II,Non-diabetic
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,Formerly smoked,0,50-59,Obese class I,Non-diabetic


In [16]:

personal_attribute_table = pd.DataFrame(df,
                  columns = ['id', 'gender','age'])
demographic_info_table = pd.DataFrame(df,
                  columns = ['id','ever_married', 'work_type','residence_type'])
health_attribute_table = pd.DataFrame(df,
                  columns = ['id','hypertension', 'heart_disease','smoking_status',
                             'avg_glucose_level','bmi'])
                             

In [17]:
personal_attribute_table = personal_attribute_table.rename({'id' : 'personal_key'},axis = 'columns')
demographic_info_table = demographic_info_table.rename({'id' : 'demo_key'},axis = 'columns')
health_attribute_table = health_attribute_table.rename({'id' : 'health_key'},axis = 'columns')

In [18]:
personal_attribute_cols = ",".join([str(i) for i in personal_attribute_table.columns.tolist()])
demographic_info_cols = ",".join([str(i) for i in demographic_info_table.columns.tolist()])
health_attribute_cols = ",".join([str(i) for i in health_attribute_table.columns.tolist()])

# Creating and Inputting into DB

In [19]:
from configparser import ConfigParser
import psycopg2
from typing import Dict


def load_connection_info(
    ini_filename: str
) -> Dict[str, str]:
    parser = ConfigParser()
    parser.read(ini_filename)
    # Create a dictionary of the variables stored under the "postgresql" section of the .ini
    conn_info = {param[0]: param[1] for param in parser.items("postgresql")}
    return conn_info


def create_db(
    conn_info: Dict[str, str],
) -> None:
    # Connect just to PostgreSQL with the user loaded from the .ini file
    psql_connection_string = f"user={conn_info['user']} password={conn_info['password']}"
    conn = psycopg2.connect(psql_connection_string)
    cur = conn.cursor()

    # "CREATE DATABASE" requires automatic commits
    conn.autocommit = True
    sql_query = f"CREATE DATABASE {conn_info['database']}"

    try:
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        cur.close()
    else:
        # Revert autocommit settings
        conn.autocommit = False


def create_table(
    sql_query: str, 
    conn: psycopg2.extensions.connection, 
    cur: psycopg2.extensions.cursor
) -> None:
    try:
        # Execute the table creation query
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        conn.rollback()
        cur.close()
    else:
        # To take effect, changes need be committed to the database
        conn.commit()

if __name__ == "__main__":
    # host, database, user, password
    conn_info = load_connection_info("db.ini")

    # Create the desired database
    create_db(conn_info)

    # Connect to the database created
    connection = psycopg2.connect(**conn_info)
    cursor = connection.cursor()

    # Create the "demographic_info" table
    demographic_info = """
        CREATE TABLE demographic_info (
            demo_key INTEGER PRIMARY KEY,
            ever_married TEXT,
            work_type TEXT,
            residence_type TEXT
        )
    """
    create_table(demographic_info, connection, cursor)



    # Create the "health_attribute" table
    health_attribute = """
        CREATE TABLE health_attribute (
            health_key INTEGER PRIMARY KEY,
            hypertension INTEGER,
            heart_disease INTEGER,
            smoking_status TEXT,
            avg_glucose_level DOUBLE PRECISION,
            bmi DOUBLE PRECISION
        )
    """
    create_table(health_attribute, connection, cursor)

    # Create the "personal_attributes" table
    personal_attributes  = """
        CREATE TABLE personal_attributes (
            personal_key INTEGER PRIMARY KEY,
            gender TEXT,
            age DOUBLE PRECISION
        )
    """
    create_table(personal_attributes, connection, cursor)

    # Create the "fact_table" table
    fact_table = """
        CREATE TABLE fact_table (
            stroke INTEGER,
            Total_strokes INTEGER,
            Paitent_count INTEGER,
            Number_smokers INTEGER,
            Bmi_level INTEGER,
            demo_key INTEGER REFERENCES demographic_info(demo_key),
            health_key INTEGER REFERENCES health_attribute(health_key),
            personal_key INTEGER REFERENCES personal_attributes(personal_key)
        )
    """
    create_table(fact_table, connection, cursor)




    # Insert DataFrame recrds one by one.
    for i,row in personal_attribute_table.iterrows():
        personal_attr_sql = "INSERT INTO personal_attributes (personal_key, gender, age) VALUES (%s, %s, %s)"
        cursor.execute(personal_attr_sql, tuple(row))
        connection.commit()
        
    for i,row in demographic_info_table.iterrows():
        demographic_info_sql = "INSERT INTO demographic_info (demo_key,ever_married,work_type,residence_type) VALUES (%s, %s, %s,%s)"
        cursor.execute(demographic_info_sql, tuple(row))
        connection.commit()

    for i,row in health_attribute_table.iterrows():
        health_attribute_sql = "INSERT INTO health_attribute (health_key,hypertension,heart_disease,smoking_status,avg_glucose_level,bmi) VALUES (%s, %s, %s,%s,%s,%s)"
        cursor.execute(health_attribute_sql, tuple(row))
        connection.commit()
        
    connection.close()
    cursor.close()
